# LangChain Basics

In [1]:
# installing the required libraries
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# !pip - installs the packages in the base environment
# pip - installs the packages in the virtual environment

In [1]:
pip show langchain

Name: langchain
Version: 0.0.190
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\sthor\anaconda3\envs\nlp\lib\site-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# upgrading langchain
pip install langchain --upgrade -q

#### Python-dotenv

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys (OpenAI, Pinecone) from .env
load_dotenv(find_dotenv(), override=True)

os.environ.get('PINECONE_API_KEY')

'c5f778ca-37d7-4a67-a01b-28afc3feee57'

### LLM Models (Wrappers): GPT-3

In [4]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [5]:
output = llm('explain quantum mechanics in one sentence')
print(output)



Quantum mechanics is a physical theory that describes the behavior of matter and energy at the atomic and subatomic level.


In [6]:
print(llm.get_num_tokens('explain quantum mechanics in one sentence'))

7


In [7]:
output = llm.generate(['... is the capital of France.', 
                   'What is the formula for the area of a circle?'])

In [8]:
print(output.generations)

[[Generation(text='\n\nParis.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where r is the radius of the circle and A is the area of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [9]:
print(output.generations[0][0].text)



Paris.


In [10]:
len(output.generations)

2

In [11]:
output = llm.generate(['Write an orignal tagline for a burger restaurant'] * 3)

In [12]:
for o in output.generations:
    print(o[0].text, end='')



"Satisfy Your Burger Cravings at Our Table!"

"Burgers So Good, They'll Make Your Taste Buds Sing!"

"Taste the Juiciness of Our Burgers - Freshly Grilled Every Time!"

### ChatModels: GPT-3.5-Turbo and GPT-4

In [13]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [14]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='explain quantum mechanics in one sentence')
]
output = chat(messages)

In [15]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf subatomarer Ebene und ermöglicht die Berechnung von Wahrscheinlichkeiten für verschiedene Ereignisse.


### Prompt Templates

In [16]:
from langchain import PromptTemplate

In [17]:
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] output_parser=None partial_variables={} template='You are an experienced virologist.\nWrite a few sentences about the following {virus} in {language}.' template_format='f-string' validate_template=True


In [18]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='HIV', language='German'))
print(output)



HIV ist ein Virus, das Immunschwäche-Virus (Human Immundefizienz Virus) heißt. Es beschädigt die Immunzellen des Körpers und macht ihn anfällig für viele andere Krankheiten. HIV wird durch sexuellen Kontakt, das Teilen von Nadeln und Bluttransfusionen übertragen. Es ist eine chronische, lebensbedrohliche Erkrankung, die ohne angemessene Behandlung zu AIDS führen kann.


### Simple Chains

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
template = '''You are an experienced virologist.
Write a few sentences about the following {virus} in {language}.'''

prompt = PromptTemplate(
    input_variables=['virus', 'language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus': 'HSV', 'language': 'french'})


In [20]:
print(output)

Le HSV, ou le virus de l'herpès simplex, est un virus à ADN qui infecte les humains. Il existe deux types principaux de HSV : le HSV-1, qui provoque généralement des lésions orales, et le HSV-2, qui est principalement responsable des infections génitales. Le HSV peut être transmis par contact direct avec une personne infectée ou par le biais de fluides corporels. Bien qu'il ne puisse pas être guéri, il existe des traitements antiviraux qui peuvent aider à réduire les symptômes et à prévenir les récidives.


### Sequential Chains

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an experienced scientist and Python programmer.
    Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)
output = overall_chain.run('softmax')





> Entering new SimpleSequentialChain chain...


def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum()
The function `softmax(x)` takes a numpy array `x` as input and returns another numpy array which contains the softmax values for each set of scores in `x`. 

The softmax function is often used in machine learning to convert a set of numerical values into a probability distribution. It assigns a probability to each value in such a way that the sum of these probabilities is equal to 1. 

The function first calculates the exponential of each value in `x` by subtracting the maximum value in `x` from each element prior to applying the exponential function. By subtracting the maximum value, the function prevents numerical instability that can occur when exponentiating large numbers.

The resulting exponential values are then divided element-wise by the sum of all exponential values in order to normalize t

### LangChain Agents

In [55]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool  
from langchain.llms import OpenAI

In [57]:
llm = OpenAI(temperature=0)
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
# agent_executor.run('Calculate the square root of the factorial of 20 \
# and display it with 4 decimal points')

agent_executor.run('what is the answer to 5.1 ** 7.3?')





> Entering new AgentExecutor chain...
 I need to use the power operator
Action: Python REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'